In [ ]:
# tag::ml_pipeline_preprocess[]
import ray
from ray.util.dask import enable_dask_on_ray

import dask.dataframe as dd

LABEL_COLUMN = "is_big_tip"
FEATURE_COLUMNS = ["passenger_count", "trip_distance", "fare_amount",
                   "trip_duration", "hour", "day_of_week"]

enable_dask_on_ray()


def load_dataset(path: str, *, include_label=True):
    columns = ["tpep_pickup_datetime", "tpep_dropoff_datetime", "tip_amount",
               "passenger_count", "trip_distance", "fare_amount"]
    df = dd.read_parquet(path, columns=columns)  # <1>

    df = df.dropna()  # <2>
    df = df[(df["passenger_count"] <= 4) &
            (df["trip_distance"] < 100) &
            (df["fare_amount"] < 1000)]

    df["tpep_pickup_datetime"] = dd.to_datetime(df["tpep_pickup_datetime"])
    df["tpep_dropoff_datetime"] = dd.to_datetime(df["tpep_dropoff_datetime"])

    df["trip_duration"] = (df["tpep_dropoff_datetime"] -
                           df["tpep_pickup_datetime"]).dt.seconds
    df = df[df["trip_duration"] < 4 * 60 * 60] # 4 hours.
    df["hour"] = df["tpep_pickup_datetime"].dt.hour
    df["day_of_week"] = df["tpep_pickup_datetime"].dt.weekday  # <3>

    if include_label:
        df[LABEL_COLUMN] = df["tip_amount"] > 0.2 * df["fare_amount"]  # <4>

    df = df.drop(  # <5>
        columns=["tpep_pickup_datetime", "tpep_dropoff_datetime", "tip_amount"]
    )

    return ray.data.from_dask(df).repartition(100)  # <6>
# end::ml_pipeline_preprocess[]

In [ ]:
# tag::ml_pipeline_model[]
import torch
import torch.nn as nn
import torch.nn.functional as F


class FarePredictor(nn.Module):
    def __init__(self):
        super().__init__()

        self.fc1 = nn.Linear(6, 256)
        self.fc2 = nn.Linear(256, 16)
        self.fc3 = nn.Linear(16, 1)

        self.bn1 = nn.BatchNorm1d(256)
        self.bn2 = nn.BatchNorm1d(16)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.bn1(x)
        x = F.relu(self.fc2(x))
        x = self.bn2(x)
        x = torch.sigmoid(self.fc3(x))

        return x
# end::ml_pipeline_model[]

In [ ]:
# tag::ml_pipeline_train_1[]
from ray.air import session
from ray.air.config import ScalingConfig
import ray.train as train
from ray.train.torch import TorchCheckpoint, TorchTrainer


def train_loop_per_worker(config: dict):  # <1>
    batch_size = config.get("batch_size", 32)
    lr = config.get("lr", 1e-2)
    num_epochs = config.get("num_epochs", 3)

    dataset_shard = train.get_dataset_shard("train")  # <2>

    model = FarePredictor()
    dist_model = train.torch.prepare_model(model)  # <3>

    loss_function = nn.SmoothL1Loss()
    optimizer = torch.optim.Adam(dist_model.parameters(), lr=lr)

    for epoch in range(num_epochs):  # <4>
        loss = 0
        num_batches = 0
        for batch in dataset_shard.iter_torch_batches(  # <5>
                batch_size=batch_size, dtypes=torch.float
        ):
            labels = torch.unsqueeze(batch[LABEL_COLUMN], dim=1)
            inputs = torch.cat(
                [torch.unsqueeze(batch[f], dim=1) for f in FEATURE_COLUMNS], dim=1
            )
            output = dist_model(inputs)
            batch_loss = loss_function(output, labels)
            optimizer.zero_grad()
            batch_loss.backward()
            optimizer.step()

            num_batches += 1
            loss += batch_loss.item()

        session.report(  # <6>
            {"epoch": epoch, "loss": loss},
            checkpoint=TorchCheckpoint.from_model(dist_model)
        )
# end::ml_pipeline_train_1[]

In [ ]:
# tag::ml_pipeline_train_2[]
trainer = TorchTrainer(
    train_loop_per_worker=train_loop_per_worker,  # <1>
    train_loop_config={  # <2>
        "lr": 1e-2, "num_epochs": 3, "batch_size": 64
    },
    scaling_config=ScalingConfig(num_workers=2),  # <3>
    datasets={  # <4>
        "train": load_dataset("nyc_tlc_data/yellow_tripdata_2020-01.parquet")
    },
)

result = trainer.fit()  # <5>
trained_model = result.checkpoint
# end::ml_pipeline_train_2[]

In [ ]:
# tag::ml_pipeline_inference[]
from ray.train.torch import TorchPredictor
from ray.train.batch_predictor import BatchPredictor

batch_predictor = BatchPredictor(trained_model, TorchPredictor)
ds = load_dataset(
    "nyc_tlc_data/yellow_tripdata_2021-01.parquet", include_label=False)

batch_predictor.predict_pipelined(ds, blocks_per_window=10)
# end::ml_pipeline_inference[]

In [ ]:
# tag::training_func_setup[]
import torch
import torch.nn as nn
from ray.data import read_datasource, datasource

num_samples = 20
input_size = 10
layer_size = 15
output_size = 5
num_epochs = 3


class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(input_size, layer_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(layer_size, output_size)

    def forward(self, x):
        x = F.relu(self.f12(x))
        x = self.fc2(x)
        return x


def train_data():
    return torch.randn(num_samples, input_size)  # <1>


input_data = train_data()
label_data = torch.randn(num_samples, output_size)

source = datasource.SimpleTorchDatasource()  # <2>
train_dataset = read_datasource(source, dataset_factory=train_data)


def train_one_epoch(model, loss_fn, optimizer):  # <3>
    output = model(input_data)
    loss = loss_fn(output, label_data)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


def training_loop():  # <4>
    model = NeuralNetwork()
    loss_fn = nn.MSELoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=0.1)
    for epoch in range(num_epochs):
        train_one_epoch(model, loss_fn, optimizer)
# end::training_func_setup[]

In [ ]:
# tag::dist_func_setup[]
from ray.train.torch import prepare_model


def distributed_training_loop():
    model = NeuralNetwork()
    model = prepare_model(model)  # <1>
    loss_fn = nn.MSELoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=0.1)
    for epoch in range(num_epochs):
        train_one_epoch(model, loss_fn, optimizer)

# end::dist_func_setup[]


# TODO: this code previously didn't work. we need to provide a "dataset" argument here.
# Notet that this is also wrong in the current docs, which state that this is possible.
# TODO running this results in: Result dict has no key: training_iteration. checkpoint_score_attr must be set to a key in the result dict. Valid keys are: ['trial_id', 'experiment_id', 'date', 'timestamp', 'pid', 'hostname', 'node_ip', 'config', 'done']

In [ ]:
# tag::train_start[]
from ray.air.config import ScalingConfig
from ray.train.torch import TorchTrainer


trainer = TorchTrainer(
    train_loop_per_worker=distributed_training_loop,
    scaling_config=ScalingConfig(
        num_workers=2,
        use_gpu=False
    ),
    datasets={"train": train_dataset}
)

result = trainer.fit()
# end::train_start[]



In [ ]:
# tag::ray_tune_integration_1[]
from ray import tune
from ray.data.preprocessors import StandardScaler, MinMaxScaler


dataset = ray.data.from_items(
    [{"X": 1.0, "Y": 2.0}, {"X": 4.0, "Y": 0.0}]
)
prep_v1 = StandardScaler(columns=["X", "Y"])
prep_v2 = MinMaxScaler(columns=["X", "Y"])

param_space = {
    "scaling_config": ScalingConfig(
        num_workers=tune.grid_search([2, 4]),
        resources_per_worker={
            "CPU": 2,
            "GPU": 0,
        },
    ),
    "preprocessor": tune.grid_search([prep_v1, prep_v2]),
    "params": {
        "objective": "binary:logistic",
        "tree_method": "approx",
        "eval_metric": ["logloss", "error"],
        "eta": tune.loguniform(1e-4, 1e-1),
        "subsample": tune.uniform(0.5, 1.0),
        "max_depth": tune.randint(1, 9),
    },
}
# end::ray_tune_integration_1[]

# tag::ray_tune_integration_2[]
from ray.train.xgboost import XGBoostTrainer
from ray.air.config import RunConfig
from ray.tune import Tuner


trainer = XGBoostTrainer(
    run_config=RunConfig(verbose=1),
    preprocessor=None,
    scaling_config=None,
    label_column="Y",
    datasets={"train": dataset}
)

tuner = Tuner(
    trainer,
    param_space=param_space,
)

results = tuner.fit()
# end::ray_tune_integration_2[]